In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/FMCG_CIG_BEER.xlsx', sheet_name = 'index + mcellname')

In [6]:
df['new_mcell_name'] = df['new_mcell_name'].str.replace('Rostov-na-Donu', 'Rostov na Donu')
df['new_mcell_name'] = df['new_mcell_name'].str.replace('Ulan-Ude', 'Ulan Ude')
df['new_mcell_name'] = df['new_mcell_name'].str.replace('Komsomolsk-na-Amure', 'Komsomolsk na Amure')

In [7]:
def parser(mcell):
    if mcell.split('-')[0] in ['CNT', 'FST', 'VOL', 'URL', 'NWT', 'STH', 'MSO', 'SIB']:
        area = [mcell.split('-')[0]]
        if 'RR' in mcell.split('-')[-1]:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            types = ['OSU', 'OLA', 'OME', 'OSM', 'OPV', 'OMM', 'OMS']
            pgr = list(mcell.split('-')[2].split('|'))
            ds = list(mcell.split('-')[3].split('|'))
            hip = []
            ndx = [mcell.split('-')[-1]]
            NO_city = []
            ttr = ['MT', 'TT']
        else:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            #if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
            #    types = ['OKS', 'ONS', 'OPV', 'OSM']
            #else:
            types = list(mcell.split('-')[2].split('|'))
            pgr = list(map(int, mcell.split('-')[3].split('|')))
            ttr = list(mcell.split('-')[4].split('|'))
            try:
                NO_city = list(map(int, mcell.split('-')[5].replace('(', '').replace(')', '').split('&')))
            except:
                NO_city = []
            hip = []
            ndx = [mcell.split('-')[-1]]
            ds = []
    else:
        if '|' in mcell.split('-')[0]:
            area = [mcell.split('-')[0] + ' ' + mcell.split('-')[1]]
        else:
            area = [mcell.split('-')[1]]
        ccodes = list(map(int, mcell.split('-')[0].split('|')))
        subreg = []
        types = list(mcell.split('-')[2].split('|'))
        if ('Y' in mcell.split('-')[3]) | ('N' in mcell.split('-')[3]):
            ttr = ['TT']
            hip = list(mcell.split('-')[3])
        else:
            ttr = list(mcell.split('-')[3].split('|'))
            hip = []
        NO_city = []
        pgr = []
        ndx = [mcell.split('-')[-1]]
        ds = []
    return(area, ccodes, subreg, types, pgr, ttr, NO_city, hip, ds, ndx)

In [8]:
res = pd.DataFrame(columns = ['index', 'area', 'ccodes', 'subreg', 'types', 'pgr', 'ttr', 'NO_city', 'hip', 'ds', 'ndx'])

In [9]:
for i in df.index:
    res.loc[i, 'index'] = df['index'][i]
    a = parser(df['new_mcell_name'][i])
    res.loc[i, 'area'] = a[0]
    res.loc[i, 'ccodes'] = a[1]
    res.loc[i, 'subreg'] = a[2]
    res.loc[i, 'types'] = a[3]
    res.loc[i, 'pgr'] = a[4]
    res.loc[i, 'ttr'] = a[5]
    res.loc[i, 'NO_city'] = a[6]
    res.loc[i, 'hip'] = a[7]
    res.loc[i, 'ds'] = a[8]
    res.loc[i, 'ndx'] = a[9]

In [10]:
res = res.set_index('index')

In [11]:
cell_rules = pd.DataFrame(columns = ['index', 'RULE_DESCRIPTION', 'RULE_TYPE', 'ATTRIBUTE_ID', 'OPERATOR', 'VALUE', 
                                     'OPENBRACE', 'CLOSEBRACE', 'CONDITION'])

In [12]:
CELL_AREA = res[['area', 'ccodes', 'subreg', 'pgr', 'ds', 'NO_city']].astype('str').drop_duplicates()

In [13]:
k = -1
for i, j in CELL_AREA.iterrows():
    RULE_DESCRIPTION = j['area'].replace('[', '').replace(']', '').replace("'", "")
    if j['ccodes'] != '[]':
        k += 1
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 20125
        if len(j['ccodes'].split(',')) == 1:
            cell_rules.loc[k, 'OPERATOR'] = '='
        else:
            cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = j['ccodes'].replace('[', '').replace(']', '').replace(', ', '|')
    else:
        ##########subreg###########
        k += 1
        RULE_DESCRIPTION += ' ' + j['subreg'].replace('[', '').replace(']', '').replace("'", "").replace(', ', '|')
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 1001023
        if len(j['subreg'].split(',')) == 1:
            cell_rules.loc[k, 'OPERATOR'] = '='
        else:
            cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = j['subreg'].replace('[', '').replace(']', '').replace(', ', '|')
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        ##########pgr##############
        k += 1
        RULE_DESCRIPTION += ' ' + j['pgr'].replace('[', '').replace(']', '').replace("'", "").replace(', ', '|')
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 1001022
        if len(j['pgr'].replace('[', '').replace(']', '').replace(', ', '|').replace("'", "").split('|')) == 1:
            cell_rules.loc[k, 'OPERATOR'] = '='
        else:
            cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = j['pgr'].replace('[', '').replace(']', '').replace(', ', '|').replace("'", "")
        ##########distance_strata###
        if j['ds'] != '[]':
            cell_rules.loc[k, 'CONDITION'] = 'AND'
            k += 1
            RULE_DESCRIPTION += ' ' + j['ds'].replace('[', '').replace(']', '').replace("'", "")
            cell_rules.loc[k, 'ATTRIBUTE_ID'] = 21118
            cell_rules.loc[k, 'OPERATOR'] = '='
            cell_rules.loc[k, 'VALUE'] = int(j['ds'].replace('[', '').replace(']', '').replace("'", ""))
        ##########NO_city###
        if j['NO_city'] != '[]':
            RULE_DESCRIPTION += ' NO ' + j['NO_city'].replace('[', '').replace(']', '').replace("'", "").replace(', ', '&')
#            cell_rules.loc[k, 'CONDITION'] = 'AND'    #########################################
            for n in range(len(j['NO_city'].replace('[', '').replace(']', '').split(','))):
                cell_rules.loc[k, 'CONDITION'] = 'AND'
                k += 1
                cell_rules.loc[k, 'ATTRIBUTE_ID'] = 20125
                cell_rules.loc[k, 'OPERATOR'] = '!='
                cell_rules.loc[k, 'VALUE'] = int(j['NO_city'].replace('[', '').replace(']', '').split(',')[n])
    cell_rules.loc[k, 'RULE_DESCRIPTION'] = RULE_DESCRIPTION
    cell_rules.loc[k, 'index'] = i
cell_rules['RULE_DESCRIPTION'] = cell_rules['RULE_DESCRIPTION'].bfill()
cell_rules['index'] = cell_rules['index'].bfill()
cell_rules['RULE_TYPE'] = 'CELL_AREA'

In [14]:
cell_area = cell_rules

In [15]:
cell_rules = pd.DataFrame(columns = ['index', 'RULE_DESCRIPTION', 'RULE_TYPE', 'ATTRIBUTE_ID', 'OPERATOR', 'VALUE', 
                                     'OPENBRACE', 'CLOSEBRACE', 'CONDITION'])

In [16]:
CELL_STORE = res[['types', 'ttr', 'hip']].astype('str').drop_duplicates()

In [17]:
k = -1
for i, j in CELL_STORE.iterrows():
    ##########types###########
    k += 1
    RULE_DESCRIPTION = j['types'].replace('[', '').replace(']', '').replace("'", "").replace(', ', '|')
    cell_rules.loc[k, 'ATTRIBUTE_ID'] = 6
    if 'IML' in j['types']:
        cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = 'ONS|OKS|OPV|OSM'
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        k += 1
        RULE_DESCRIPTION = j['types'].replace('[', '').replace(']', '').replace("'", "")
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 15
        cell_rules.loc[k, 'OPERATOR'] = '>'
        cell_rules.loc[k, 'VALUE'] = 12
        cell_rules.loc[k, 'CONDITION'] = 'AND'
    elif 'IMS' in j['types']:
        cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = 'ONS|OKS|OPV|OSM'
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        k += 1
        RULE_DESCRIPTION = j['types'].replace('[', '').replace(']', '').replace("'", "")
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 15
        cell_rules.loc[k, 'OPERATOR'] = '<='
        cell_rules.loc[k, 'VALUE'] = 12
        cell_rules.loc[k, 'CONDITION'] = 'AND'
    else:
        if len(j['types'].split(',')) == 1:
            cell_rules.loc[k, 'OPERATOR'] = '='
        else:
            cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = j['types'].replace('[', '').replace(']', '').replace(', ', '|').replace("'", "")
        cell_rules.loc[k, 'CONDITION'] = 'AND'
    ##########ttr##############
    k += 1
    RULE_DESCRIPTION += ' ' + j['ttr'].replace('[', '').replace(']', '').replace(', ', '|').replace("'", "")
    cell_rules.loc[k, 'ATTRIBUTE_ID'] = 24
    if len(j['ttr'].split(',')) == 1:
        cell_rules.loc[k, 'OPERATOR'] = '='
    else:
        cell_rules.loc[k, 'OPERATOR'] = 'IN'
    cell_rules.loc[k, 'VALUE'] = j['ttr'].replace('[', '').replace(']', '').replace(', ', '|').replace("'", "")
    ##########hip###
    if j['hip'] != '[]':
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        k += 1
        RULE_DESCRIPTION += ' ' + j['hip'].replace('[', '').replace(']', '').replace("'", "")
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 9227
        cell_rules.loc[k, 'OPERATOR'] = '='
        cell_rules.loc[k, 'VALUE'] = j['hip'].replace('[', '').replace(']', '').replace("'", "")
    ######RULE_DESCRIPTION########
    cell_rules.loc[k, 'RULE_DESCRIPTION'] = RULE_DESCRIPTION
    cell_rules.loc[k, 'index'] = i
cell_rules['RULE_DESCRIPTION'] = cell_rules['RULE_DESCRIPTION'].bfill()
cell_rules['index'] = cell_rules['index'].bfill()
cell_rules['RULE_TYPE'] = 'CELL_STORE'

In [18]:
cell_store = cell_rules

In [19]:
cell_rules = pd.DataFrame(columns = ['index', 'RULE_DESCRIPTION', 'RULE_TYPE', 'ATTRIBUTE_ID', 'OPERATOR', 'VALUE', 
                                     'OPENBRACE', 'CLOSEBRACE', 'CONDITION'])

In [20]:
MICRO_CELL = res.astype('str')

In [21]:
k = -1
for i, j in MICRO_CELL.iterrows():
    if j['ccodes'] != '[]':
        k += 1
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 20125
        if len(j['ccodes'].split(',')) == 1:
            cell_rules.loc[k, 'OPERATOR'] = '='
        else:
            cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = j['ccodes'].replace('[', '').replace(']', '').replace(', ', '|')
    else:
        #subreg
        k += 1
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 1001023
        if len(j['subreg'].split(',')) == 1:
            cell_rules.loc[k, 'OPERATOR'] = '='
        else:
            cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = j['subreg'].replace('[', '').replace(']', '').replace(', ', '|')
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        #pgr
        k += 1
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 1001022
        if len(j['pgr'].replace('[', '').replace(']', '').replace(', ', '|').replace("'", "").split('|')) == 1:
            cell_rules.loc[k, 'OPERATOR'] = '='
        else:
            cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = j['pgr'].replace('[', '').replace(']', '').replace(', ', '|').replace("'", "")
        #distance_strata
        if j['ds'] != '[]':
            cell_rules.loc[k, 'CONDITION'] = 'AND'
            k += 1
            cell_rules.loc[k, 'ATTRIBUTE_ID'] = 21118
            cell_rules.loc[k, 'OPERATOR'] = '='
            cell_rules.loc[k, 'VALUE'] = int(j['ds'].replace('[', '').replace(']', '').replace("'", ""))
        #NO_city
        if j['NO_city'] != '[]':
#            cell_rules.loc[k, 'CONDITION'] = 'AND'
            for n in range(len(j['NO_city'].replace('[', '').replace(']', '').split(','))):
                cell_rules.loc[k, 'CONDITION'] = 'AND' 
                k += 1
                cell_rules.loc[k, 'ATTRIBUTE_ID'] = 20125
                cell_rules.loc[k, 'OPERATOR'] = '!='
                cell_rules.loc[k, 'VALUE'] = int(j['NO_city'].replace('[', '').replace(']', '').split(',')[n])
    cell_rules.loc[k, 'CONDITION'] = 'AND'
    k += 1
    cell_rules.loc[k, 'ATTRIBUTE_ID'] = 6
    if 'IML' in j['types']:
        cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = 'ONS|OKS|OPV|OSM'
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        k += 1
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 15
        cell_rules.loc[k, 'OPERATOR'] = '>'
        cell_rules.loc[k, 'VALUE'] = 12
        cell_rules.loc[k, 'CONDITION'] = 'AND'
    elif 'IMS' in j['types']:
        cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = 'ONS|OKS|OPV|OSM'
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        k += 1
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 15
        cell_rules.loc[k, 'OPERATOR'] = '<='
        cell_rules.loc[k, 'VALUE'] = 12
        cell_rules.loc[k, 'CONDITION'] = 'AND'
    else:
        if len(j['types'].split(',')) == 1:
            cell_rules.loc[k, 'OPERATOR'] = '='
        else:
            cell_rules.loc[k, 'OPERATOR'] = 'IN'
        cell_rules.loc[k, 'VALUE'] = j['types'].replace('[', '').replace(']', '').replace(', ', '|').replace("'", "")
        cell_rules.loc[k, 'CONDITION'] = 'AND'
    #ttr
    k += 1
    cell_rules.loc[k, 'ATTRIBUTE_ID'] = 24
    if len(j['ttr'].split(',')) == 1:
        cell_rules.loc[k, 'OPERATOR'] = '='
    else:
        cell_rules.loc[k, 'OPERATOR'] = 'IN'
    cell_rules.loc[k, 'VALUE'] = j['ttr'].replace('[', '').replace(']', '').replace(', ', '|').replace("'", "")
    #hip
    if j['hip'] != '[]':
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        k += 1
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 9227
        cell_rules.loc[k, 'OPERATOR'] = '='
        cell_rules.loc[k, 'VALUE'] = j['hip'].replace('[', '').replace(']', '').replace("'", "")  
    #NOTNEEDCHA NEEDCCODE RUCHA
    cell_rules.loc[k, 'CONDITION'] = 'AND'
    k += 1
    cell_rules.loc[k, 'ATTRIBUTE_ID'] = 21113
    cell_rules.loc[k, 'OPERATOR'] = '='
    cell_rules.loc[k, 'VALUE'] = 1
    cell_rules.loc[k, 'CONDITION'] = 'AND'
    k += 1
    cell_rules.loc[k, 'ATTRIBUTE_ID'] = 1001033
    cell_rules.loc[k, 'OPERATOR'] = '='
    cell_rules.loc[k, 'VALUE'] = 'OTHER'
    cell_rules.loc[k, 'CONDITION'] = 'AND'
    k += 1
    cell_rules.loc[k, 'ATTRIBUTE_ID'] = 21114
    cell_rules.loc[k, 'OPERATOR'] = '='
    cell_rules.loc[k, 'VALUE'] = 1
    #HL CIG BEER
    if  ((3999 < i) & (i < 6000)) | ((10000 < i) & (i < 11000)):
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        k += 1
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 8517
        cell_rules.loc[k, 'OPERATOR'] = '='
        cell_rules.loc[k, 'VALUE'] = 1
    if  ((5999 < i) & (i < 7000)) | (11000 < i):
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        k += 1
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 8514
        cell_rules.loc[k, 'OPERATOR'] = '='
        cell_rules.loc[k, 'VALUE'] = 1
    #sms service
    if  10000 <= i:
        cell_rules.loc[k, 'CONDITION'] = 'AND'
        k += 1
        cell_rules.loc[k, 'ATTRIBUTE_ID'] = 39413
        cell_rules.loc[k, 'OPERATOR'] = '='
        cell_rules.loc[k, 'VALUE'] = 1    
    cell_rules.loc[k, 'index'] = i
cell_rules['index'] = cell_rules['index'].bfill()
cell_rules['RULE_TYPE'] = 'MICRO_CELL'

In [22]:
for i in df.index:
    cell_rules.loc[cell_rules['index'] == df['index'][i], 'RULE_DESCRIPTION'] = df['new_mcell_name'][i]

In [23]:
res = cell_rules.append(cell_store, sort = False).append(cell_area, sort = False)

In [24]:
res = res[['RULE_DESCRIPTION', 'RULE_TYPE', 'ATTRIBUTE_ID', 'OPERATOR', 'VALUE', 'OPENBRACE', 'CLOSEBRACE', 'CONDITION']]

In [25]:
res.to_csv('rules_maxim.csv', sep = ',', index = False)

In [26]:
res.to_excel('rules_maxim.xlsx', index = False)

# UAT

In [29]:
res_uat = res[['RULE_DESCRIPTION', 'RULE_TYPE', 'ATTRIBUTE_ID', 'OPERATOR', 'VALUE', 'OPENBRACE', 'CLOSEBRACE', 'CONDITION']]

In [30]:
res_uat.loc[res_uat['ATTRIBUTE_ID'] == 20125, 'ATTRIBUTE_ID'] = 20566
res_uat.loc[res_uat['ATTRIBUTE_ID'] == 21118, 'ATTRIBUTE_ID'] = 203051
res_uat.loc[res_uat['ATTRIBUTE_ID'] == 9227, 'ATTRIBUTE_ID'] = 173184
res_uat.loc[res_uat['ATTRIBUTE_ID'] == 8517, 'ATTRIBUTE_ID'] = 128036
res_uat.loc[res_uat['ATTRIBUTE_ID'] == 8514, 'ATTRIBUTE_ID'] = 128032
res_uat.loc[res_uat['ATTRIBUTE_ID'] == 21113, 'ATTRIBUTE_ID'] = 203046
res_uat.loc[res_uat['ATTRIBUTE_ID'] == 1001033, 'ATTRIBUTE_ID'] = 173163
res_uat.loc[res_uat['ATTRIBUTE_ID'] == 21114, 'ATTRIBUTE_ID'] = 203047

In [31]:
res_uat['RULE_DESCRIPTION'] = 'max_max ' + res_uat['RULE_DESCRIPTION'] 

In [32]:
res_uat.to_csv('rules_maxim_uat.csv', sep = ',', index = False)

In [33]:
res_uat.to_excel('rules_maxim_uat.xlsx', index = False)